In [1]:
import os
import torch
import pandas as pd
import numpy as np

from Load_Data import random_seed
from Train import Trainer
from Inference import Model_Ensemble, run_inference

In [2]:
# 시드 고정
random_seed(42)

# 기본 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
img_size = 384
LR = 1e-5
EPOCHS = 60
BATCH_SIZE = 12
Drop_out = 0.4
weight_decay = 5e-4
num_workers = 4
num_classes = 17
n_splits = 3
patience = 5
model_name = "convnextv2_large.fcmae_ft_in22k_in1k_384"
# model_name = "convnextv2_huge.fcmae_ft_in22k_in1k_512"
# model_name = "convnext_large_in22ft1k" # batch_size=14~16
# model_name = "convnext_base_in22ft1k" # batch_size=32
# model_name = "maxvit_large_tf_384"

# 경로
run_name = "V18"
train_csv_path = "../data/train_update2.csv"
test_csv_path = "../data/sample_submission.csv"
original_train_path = "../data/train/"
test_path = "../data/test/"
model_save_path = f"../model/{run_name}/"
augmented_save_path = f"../data/augment_image/{run_name}-augmented"
augmented_csv_save_path = f"../data/augment_csv/{run_name}-augmented.csv"
submission_path = f"../data/submission/{run_name}-submission.csv"

In [ ]:
# 원본 데이터
train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)

# Trainer 생성
trainer = Trainer(
    df=train_df,
    original_data_path=original_train_path,
    augmented_save_path=augmented_save_path,
    augmented_csv_save_path=augmented_csv_save_path,
    model_name=model_name,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    lr=LR,
    drop_out=Drop_out,
    weight_decay=weight_decay,
    img_size=img_size,
    num_workers=num_workers,
    device=device,
    save_dir=model_save_path,
    run_name_prefix=run_name,
    num_classes=num_classes,
    n_splits=n_splits,
    patience=patience,
    k_fold=True,
    augmentation_target_count=450
)

# K-Fold 분할, 증강, 학습 모두 실행
f1_df = trainer.run()

In [ ]:
# 앙상블 모델
# fold_weights = np.array([0.9416, 0.94768, 0.93204])
fold_weights = f1_df["f1"].values
ensembler = Model_Ensemble(
    model_name=model_name,
    fold_paths_dir=model_save_path,
    fold_weights=fold_weights,
    num_classes=num_classes,
    drop_out=Drop_out,
    device=device,
    k_fold=True
)

# 추론 실행 (TTA 적용)
run_inference(
    ensembler=ensembler,
    submission_df=test_df.copy(),
    test_path=test_path,
    img_size=img_size,
    save_path=submission_path, # 저장
    batch_size=BATCH_SIZE,
    num_workers=num_workers,
    use_tta=True # TTA 
)

In [5]:
test_df = pd.read_csv(test_csv_path)
model_groups = [
    {
        'version': 'V15', 
        'path_dir': '../model/V15',
        'model_name': 'convnext_xlarge_in22ft1k', 
        'img_size': 512, 
        'group_weight': 0.7  # V15 그룹 전체에 70%의 가중치 할당
    },
    {
        'version': 'V18', 
        'path_dir': '../model/V18',
        'model_name': 'convnextv2_large.fcmae_ft_in22k_in1k_384', 
        'img_size': 384, 
        'group_weight': 0.3  # V18 그룹 전체에 30%의 가중치 할당
    }
]

ensembler = Model_Ensemble(
    model_groups=model_groups,
    num_classes=num_classes,
    drop_out=Drop_out,
    device=device
)

run_inference(
    ensembler=ensembler,
    submission_df=test_df.copy(),
    test_path=test_path,
    save_path=submission_path, # 최종 제출 경로
    batch_size=1, # 여러 모델을 처리하므로 배치 사이즈를 작게 설정
    num_workers=num_workers,
    use_tta=True
)

Loading 3 models from 'V15' group with weight: 0.2333 each...
Loading 2 models from 'V18' group with weight: 0.1500 each...


Inference: 100%|██████████| 3140/3140 [46:58<00:00,  1.11it/s]

[✓] Saved submission to: ../data/submission/V18-submission.csv
